# Final Project: Face Mask Detection
Welcome to the final project assignment. For this project, we will be working on the Face Mask detection problem.

The Dataset contains images of people wearing masks and people not wearing masks. The database contains 10,000 colored images in the training folder, 800 images in the validation folder, and 992 images in the test folder.

Your task is to create a CNN model for identifying whether a person in the image is wearing a mask or not.

Please perform the following tasks:

Download the database, click here to download it.
Create test, train, and validation directory variables
Create train and validation data generator with target size (128,128)
Train a CNN model
Train a model with VGG19 model
Use callbacks to save your model at every step
Training may take several hours, so use 5 to 10 epochs only

In [1]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import os

In [3]:
import tensorflow as tf
from tensorflow import keras

In [4]:
# Two folders are present inside each full path directory which contain the images
train_dir = r'C:\Users\Deeshu\Desktop\Face Mask Detection Using Deep Learning\Datasets\Train'
validation_dir =r'C:\Users\Deeshu\Desktop\Face Mask Detection Using Deep Learning\Datasets\Validation'
test_dir =r'C:\Users\Deeshu\Desktop\Face Mask Detection Using Deep Learning\Datasets\Test'

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [6]:
train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(128, 128),
        batch_size=20,
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(128, 128),
        batch_size=20,
        class_mode='binary')

Found 10000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


In [7]:
# Importing the VGG19 to use its Conv Base
from tensorflow.keras.applications import VGG19
# weights =  we are using the final weights (pre trained weights)
# imagenet is ILSVRC Competition so its weights
# include top = false, because we are only using the conv base
# If wanted to import the whole model then include_top=True
# input shape = normal input shape of our images {target size + channels}
conv_base = VGG19(weights='imagenet',
                  include_top=False,
                  input_shape=(128, 128, 3))

In [8]:
conv_base.summary()

Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 128, 128, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 128, 128, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 64, 64, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 64, 64, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 64, 64, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 32, 32, 128)       0     

In [9]:
from tensorflow.keras import models
from tensorflow.keras import layers

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 4, 4, 512)         20024384  
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 256)               2097408   
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 22,122,049
Trainable params: 22,122,049
Non-trainable params: 0
_________________________________________________________________


In [11]:
from tensorflow.keras import optimizers
# using small learning rate becoz we want to fine tune our model
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(learning_rate=2e-5),
              metrics=['acc'])

In [12]:
checkpoint_cb = keras.callbacks.ModelCheckpoint("Face_Mask_Detect_Model-{epoch:02d}.h5",save_best_only=True)

In [13]:
history = model.fit(
      train_generator,
      steps_per_epoch=100,
      epochs=10,
      validation_data=validation_generator,
      validation_steps=50,
      callbacks=[checkpoint_cb])

Epoch 1/10
100/100 [==============================] - 872s 9s/step - loss: 0.1796 - acc: 0.9245 - val_loss: 0.0244 - val_acc: 0.9925
Epoch 2/10
100/100 [==============================] - 1122s 11s/step - loss: 0.0589 - acc: 0.9800
Epoch 3/10
100/100 [==============================] - 513s 5s/step - loss: 0.0534 - acc: 0.9865
Epoch 4/10
100/100 [==============================] - 430s 4s/step - loss: 0.0309 - acc: 0.9895
Epoch 5/10
100/100 [==============================] - 436s 4s/step - loss: 0.0300 - acc: 0.9900
Epoch 6/10
100/100 [==============================] - 457s 5s/step - loss: 0.0309 - acc: 0.9910
Epoch 7/10
100/100 [==============================] - 450s 5s/step - loss: 0.0368 - acc: 0.9875
Epoch 8/10
100/100 [==============================] - 498s 5s/step - loss: 0.0279 - acc: 0.9920
Epoch 9/10
100/100 [==============================] - 481s 5s/step - loss: 0.0224 - acc: 0.9920
Epoch 10/10
100/100 [==============================] - 440s 4s/step - loss: 0.0113 - acc: 0.9960


In [16]:
history.history

{'loss': [0.1795806884765625,
  0.05893730744719505,
  0.053403038531541824,
  0.030874839052557945,
  0.029992982745170593,
  0.030900919809937477,
  0.03677445650100708,
  0.027941972017288208,
  0.02235162816941738,
  0.011336948722600937],
 'acc': [0.9244999885559082,
  0.9800000190734863,
  0.9865000247955322,
  0.9894999861717224,
  0.9900000095367432,
  0.9909999966621399,
  0.987500011920929,
  0.9919999837875366,
  0.9919999837875366,
  0.9959999918937683],
 'val_loss': [0.024365201592445374],
 'val_acc': [0.9925000071525574]}

In [17]:
test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

Found 992 images belonging to 2 classes.


In [18]:
model.evaluate(test_generator, steps=50)

50/50 [==============================] - 43s 855ms/step - loss: 0.0134 - acc: 0.9960


[0.013427396304905415, 0.9959677457809448]

In [ ]:
#99.5% Accuracy Achieved...